In [1]:
# 层和块
# 首先，回顾一下多层感知机
import torch
from torch import nn
from torch.nn import functional as F

# nn.Sequential定义了一种特殊的module
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)  # 2是批量大小，20是输入维度
net(X)

tensor([[-0.1307, -0.1621,  0.0563,  0.0067, -0.0393,  0.4039,  0.1129,  0.2276,
          0.1087, -0.4434],
        [-0.0879, -0.0079,  0.1363, -0.0801,  0.0476,  0.3674, -0.0695, -0.1085,
          0.0549, -0.3703]], grad_fn=<AddmmBackward0>)

In [2]:
# 自定义块
# 任何一个层和神经网络都应该是Module的子类
class MLP(nn.Module):  # 继承nn.Module
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
# 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层
net = MLP()
net(X)

tensor([[-0.0090,  0.0167, -0.1514,  0.0136,  0.1167, -0.1027, -0.0884, -0.0885,
         -0.2495, -0.4189],
        [-0.1794,  0.0414, -0.0170, -0.0791,  0.2305,  0.0356, -0.1084,  0.0038,
         -0.1701, -0.2835]], grad_fn=<AddmmBackward0>)

In [4]:
# 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block  # ordered dictionary
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0983,  0.2742, -0.1252, -0.1045,  0.0782, -0.0085,  0.0452,  0.1748,
          0.1173, -0.1434],
        [-0.1846,  0.3774, -0.0498, -0.1014,  0.0585, -0.1369, -0.0814,  0.0949,
          0.0398,  0.0561]], grad_fn=<AddmmBackward0>)

In [6]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.3452, grad_fn=<SumBackward0>)

In [7]:
# 混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2239, grad_fn=<SumBackward0>)